In [1]:
import nltk
import random
# from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from nltk.tokenize import word_tokenize
import pickle
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC



In [2]:
from nltk.classify import ClassifierI
from statistics import mode #Needed for voting purpose

In [3]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes) #returns the mode votes which is the the most popular
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf #passes confidence of the votes chosen

In [4]:
# documents=[(list(movie_reviews.words(fileid)),category) #It takes words as features overall
#             for category in movie_reviews.categories() #for separating categories from the file
#             for fileid in movie_reviews.fileids(category)]

In [5]:
# random.shuffle(documents)

In [6]:
# print(documents[1])

In [7]:
# all_words =[]
# for w in movie_reviews.words():
#     all_words.append(w.lower()) #THis is used to normalize the data so that all the words are hence in lower case

In [8]:
#above 4 lines commentwed for getting an external dataset

In [9]:
short_pos = open("positive.txt","r").read()
short_neg = open("negative.txt","r").read()

all_words = []
documents = []


#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]

for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

    
for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())



save_documents = open("pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()


In [10]:
all_words=nltk.FreqDist(all_words)

In [11]:
# print(all_words.most_common(15)) used to check most common used words

In [12]:
# print(all_words["stupid"])#used to check the frequency of a particular word in the document

In [13]:
word_features = list(all_words.keys())[:5000]
save_word_features = open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()


In [14]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features #this will find whether the top 5000 words are positive or negative reviews

In [15]:
# print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))


In [16]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)
#print(len(featuresets)) #used to check the size of the set

In [17]:
training_set=featuresets[:10000]
testing_set=featuresets[10000:]

In [18]:
classifier = nltk.NaiveBayesClassifier.train(training_set) #used down below with other nbs algos


In [19]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)


Classifier accuracy percent: 72.59036144578313


In [20]:
classifier.show_most_informative_features(15)


Most Informative Features
              engrossing = True              pos : neg    =     21.0 : 1.0
                 routine = True              neg : pos    =     16.3 : 1.0
                mediocre = True              neg : pos    =     15.0 : 1.0
                    flat = True              neg : pos    =     15.0 : 1.0
               inventive = True              pos : neg    =     14.3 : 1.0
                  boring = True              neg : pos    =     14.1 : 1.0
                 generic = True              neg : pos    =     13.7 : 1.0
                powerful = True              pos : neg    =     13.0 : 1.0
                  unique = True              pos : neg    =     13.0 : 1.0
                    warm = True              pos : neg    =     12.6 : 1.0
               wonderful = True              pos : neg    =     12.6 : 1.0
                haunting = True              pos : neg    =     12.3 : 1.0
              refreshing = True              pos : neg    =     12.3 : 1.0

In [21]:
# import pickle

In [22]:
# save_classifier = open("naivebayes.pickle","wb")
# pickle.dump(classifier, save_classifier)
# save_classifier.close()

In [23]:
# classifier_f = open("naivebayes.pickle", "rb")
# classifier = pickle.load(classifier_f)
# classifier_f.close()
# #this is used to read the file from pickle

In [24]:
# MNB_classifier = SklearnClassifier(MultinomialNB())
# MNB_classifier.train(training_set)
# print("MultinomialNB accuracy percent:",nltk.classify.accuracy(MNB_classifier, testing_set))

# BNB_classifier = SklearnClassifier(BernoulliNB())
# BNB_classifier.train(training_set)
# print("BernoulliNB accuracy percent:",nltk.classify.accuracy(BNB_classifier, testing_set))

In [26]:
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
# classifier.show_most_informative_features(15) # used for showing values kinda of ratio of neg to pos
save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()


MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)
save_classifier = open("pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)
save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)
save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()


SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)
save_classifier = open("pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDClassifier_classifier, save_classifier)
save_classifier.close()

# SVC_classifier = SklearnClassifier(SVC())
# SVC_classifier.train(training_set)
# print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)
save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

# NuSVC_classifier = SklearnClassifier(NuSVC())
# NuSVC_classifier.train(training_set)
# print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

Original Naive Bayes Algo accuracy percent: 72.59036144578313
MNB_classifier accuracy percent: 72.89156626506023
BernoulliNB_classifier accuracy percent: 72.7409638554217
LogisticRegression_classifier accuracy percent: 72.13855421686746
SGDClassifier_classifier accuracy percent: 70.63253012048193
LinearSVC_classifier accuracy percent: 70.48192771084338


In [27]:
voted_classifier = VoteClassifier(classifier,
                                  LinearSVC_classifier,
                                  SGDClassifier_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

voted_classifier accuracy percent: 72.7409638554217
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0


In [28]:
def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats)